# 3.1.3 代⼊を導⼊することのコスト

setを使う場合

In [1]:
; setを使ったwithdraw(簡易版)
(define (make-simplified-withdraw balance)
  (lambda (amount)
    (set! balance (- balance amount))
    balance))

 make-simplified-withdraw


In [2]:
(define W (make-simplified-withdraw 25))

 W


In [3]:
(W 20)
(W 10)

 5
 -5


set! を使わない次の make-decrementer ⼿続き

続けて呼び出しても、make-simplified-withdraw のような
集積効果はない

In [4]:
(define (make-decrementer balance)
  (lambda (amount)
    (- balance amount)))

 make-decrementer


In [5]:
(define D (make-decrementer 25))

 D


In [6]:
(D 20)
(D 10)

 5
 15


## 同⼀性と変化

In [7]:
(define D1 (make-decrementer 25))

 D1


In [8]:
(define D2 (make-decrementer 25))

 D2


In [9]:
(D1 20)
(D1 20)
(D2 20)

 5
 5
 5


上の２つは同じだが、下の２つは異なる

In [10]:
(define W1 (make-simplified-withdraw 25))

 W1


In [11]:
(define W2 (make-simplified-withdraw 25))

 W2


In [12]:
(W1 20)
(W1 20)
(W2 20)

 5
 -15
 5


式の値を変化させることなく、式の中で “等しいものは等しいものによって置き換えることができる  
という概念をサポートしている⾔語は、**参照透明(referentially transparent) である**と⾔わる。  
set! を含めたときに、参照透明性はられる。  

## 命令型プログラミングの落とし⽳

関数型プログラミングとは対照的に、  
代⼊を多⽤するプログラミングは**命令型プログラミング **という。

### 階乗を例に

In [13]:
; 例: 関数型的に
(define (factorial n)
  (define (iter product counter)
    (if (> counter n)
        product
        (iter (* counter product) (+ counter 1))))
  (iter 1 1))
(factorial 3)

 factorial
 6


In [14]:
; 命令形スタイルでの階乗
(define (factorial n)
  (let 
    (
        (product 1)
        (counter 1)
    )
    (define (iter)
      (if (> counter n)
          product
          (begin 
           (set! product (* counter product))
           (set! counter (+ counter 1))
           (iter))))
    (iter)))
(factorial 3)

 factorial
 6


これは動くけれど

```
(set! counter (+ counter 1))
(set! product (* counter product)
```
の順番を間違えたら動かなくなる 

当たり前・・・と思うけれど、この問題は、関数型プログラミングでは最初から起こらない。

## Ex 3.7

: 練習問題 3.3で述べた、パスワード機能を追加したmake-account で作った銀⾏⼝座オブジェクトについて考える。  
私たちの銀⾏システムでは、共同⼝座を作る能⼒が必要だとする。
これを実現する⼿続き make-joint を定義せよ。

make-joint は三つの引数を取る。

- ⼀つ⽬はパスワード保護された⼝座である。
- ⼆つ⽬の引数は⼝座を定義したときのパスワードと⼀致している必要があり、そうでなければ make-joint 演算を進めることはできな
い。
- 三番⽬の引数は新しいパスワードである。make-joint は、元の⼝座に新しいパスワードでもアクセスできるようにする。例えば、peter-acc が open-sesame というパスワードを持つ銀⾏⼝座だとすると、
```
(define paul-acc
    (make-joint peter-acc 'open-sesame 'rosebud))
```
このようにすることで、paul-acc という名前と rosebud というパスワードによって peter-acc に対する取引ができるようにする。

### 3.3での実装からコピー

In [15]:
; 3.3より
(define (make-account password balance)
  (define correct-password (list password))
  ;; 引き出し
  (define (withdraw amount)
    (if (>= balance amount)
      (begin 
        (set! balance (- balance amount))
        balance)
    "Insufficient funds"))

  ;; 預金
  (define (deposit amount)
    (set! balance (+ balance amount))　balance)
  
  ;; パスワード設定
  (define (add-password new-password)
    (begin
        (set! correct-password (cons new-password correct-password))
    dispatch))
  
  ;; password check
  (define (is-correct-password? password)
    (define (itr corrects password)
      (cond
       ((null? corrects) #f)
       ((eq? (car corrects) password) #t)
       (else (itr (cdr corrects) password))))
    (itr correct-password password)
  )
  
  ;;
  (define (dispatch password m)
    (if 
      (is-correct-password? password)
      (cond 
        ((eq? m 'withdraw) withdraw)
        ((eq? m 'deposit) deposit)
        ((eq? m 'new-password) add-password)
        (else (error "Unknown request: MAKE-ACCOUNT" m)))
      (lambda (args) "Incorrect password"))
    )
dispatch)

 make-account


In [16]:
;; 共有口座
(define (make-joint account password new-password)
  ((account password 'new-password) new-password))


 make-joint


In [17]:
; 口座1
(define account (make-account 'nyan 0))


 account


In [18]:
; 口座2
(define new-account (make-joint account 'nyan 'nyanko))

 new-account


In [19]:
((account 'nyan 'withdraw) 10)
((account 'nyan 'deposit) 10)
((account 'nyan 'withdraw) 10)
((new-account 'nyan 'deposit) 10)
((new-account 'nyanko 'deposit) 10)

 "Insufficient funds"
 10
 0
 10
 20


共有できてそう?
